<a href="https://colab.research.google.com/github/yazeedmshayekh2/ShAi_Training/blob/main/Task_6_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](logo1.jpg)

# **shAI Training 2021 | Level 1**

## Task #6 (End-to-End ML Project {part_2})

## Welcome to the exercises for reviewing second part of end to end ML project.
**Make sure that you read and understand ch2 from the hands-on ML book (page 72 to the end of the chapter ) before start with this notebook.**

**If you stuck with anything reread that part from the book and feel free to ask about anything in the messenger group as you go along.**

 ## Good Luck : )

## first run the following cell for the first part of the project to continue your work 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [2]:
import os
import tarfile
import urllib
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
def load_housing_data(housing_path=HOUSING_PATH):
   csv_path = os.path.join(housing_path, "housing.csv")
   return pd.read_csv(csv_path)
   
fetch_housing_data()
housing = load_housing_data()

rooms_ix, bedrooms_ix, population_ix, household_ix = [
    list(housing.columns).index(col)
    for col in ("total_rooms", "total_bedrooms", "population", "households")]

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
housing = train_set.drop("median_house_value", axis=1)
housing_labels = train_set["median_house_value"].copy()

housing_num = housing.drop("ocean_proximity", axis=1)
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
 ('imputer', SimpleImputer(strategy="median")),
 ('attribs_adder', CombinedAttributesAdder()),
 ('std_scaler', StandardScaler())])

full_pipeline = ColumnTransformer([
 ("num", num_pipeline, num_attribs),
 ("cat", OneHotEncoder(), cat_attribs)])

housing_prepared = full_pipeline.fit_transform(housing)

In [3]:
housing

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
14196,-117.03,32.71,33.0,3126.0,627.0,2300.0,623.0,3.2596,NEAR OCEAN
8267,-118.16,33.77,49.0,3382.0,787.0,1314.0,756.0,3.8125,NEAR OCEAN
17445,-120.48,34.66,4.0,1897.0,331.0,915.0,336.0,4.1563,NEAR OCEAN
14265,-117.11,32.69,36.0,1421.0,367.0,1418.0,355.0,1.9425,NEAR OCEAN
2271,-119.80,36.78,43.0,2382.0,431.0,874.0,380.0,3.5542,INLAND
...,...,...,...,...,...,...,...,...,...
11284,-117.96,33.78,35.0,1330.0,201.0,658.0,217.0,6.3700,<1H OCEAN
11964,-117.43,34.02,33.0,3084.0,570.0,1753.0,449.0,3.0500,INLAND
5390,-118.38,34.03,36.0,2101.0,569.0,1756.0,527.0,2.9344,<1H OCEAN
860,-121.96,37.58,15.0,3575.0,597.0,1777.0,559.0,5.7192,<1H OCEAN


In [4]:
housing_labels

14196    103000.0
8267     382100.0
17445    172600.0
14265     93400.0
2271      96500.0
           ...   
11284    229200.0
11964     97800.0
5390     222100.0
860      283500.0
15795    325000.0
Name: median_house_value, Length: 16512, dtype: float64

# 1- Select and Train a Model

# Let’s first train a LinearRegression model 

In [5]:
# CODE HERE
from sklearn.linear_model import LinearRegression

linear_reg = LinearRegression()
linear_reg.fit(housing_prepared, housing_labels)

LinearRegression()

# First try it out on a few instances from the training set:


In [6]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]

In [7]:
some_labels

14196    103000.0
8267     382100.0
17445    172600.0
14265     93400.0
2271      96500.0
Name: median_house_value, dtype: float64

In [8]:
# CODE HERE
subset_to_train = full_pipeline.transform(some_data)
subset_of_predictions= pd.Series(linear_reg.predict(subset_to_train))
subset_of_predictions

0    181746.543596
1    290558.749735
2    244957.500178
3    146498.510614
4    163230.423939
dtype: float64

# measure this regression model’s RMSE on the whole training set 
* sing Scikit-Learn’s mean_squared_error() function:

In [9]:
from sklearn.metrics import mean_squared_error

In [10]:
rmse_on_subset= np.sqrt(mean_squared_error(some_labels, subset_of_predictions))
rmse_on_subset

73605.47230869903

In [11]:
# CODE HERE
predictions_linear= linear_reg.predict(housing_prepared)
RMSE_linear = np.sqrt(mean_squared_error(housing_labels, predictions_linear))
RMSE_linear

67593.20745775253

# judge on the RMSE result for this model 
write down your answar 

In [12]:
housing_labels.describe()

count     16512.000000
mean     207194.693738
std      115622.626448
min       14999.000000
25%      119800.000000
50%      179850.000000
75%      265125.000000
max      500001.000000
Name: median_house_value, dtype: float64

your answer goes here

As we see here , the Rout mean squared error \$67593 this is not the best result but not the worst, because the most median_house_values are between \$119800 
and $265125, for example as we see before when we compare the values in some_labels and subset_of_predictions if we take the first value we have \$103000 as an actual value and the predictive value was \$181746, this is definitely not the best, and if we take anothers samples we're going to find the same thing so this is not the best model, let's try a more complex model.

# Let’s train a Decision Tree Regressor model 
## more powerful model

In [13]:
from sklearn.tree import DecisionTreeRegressor 

In [14]:
# CODE HERE

decision_tree = DecisionTreeRegressor()
decision_tree.fit(housing_prepared, housing_labels)

DecisionTreeRegressor()

# Now evaluate the model on the training set 
* using Scikit-Learn’s mean_squared_error() function:

In [15]:
subset_of_predictions_tree= pd.Series(decision_tree.predict(subset_to_train))
subset_of_predictions_tree

0    103000.0
1    382100.0
2    172600.0
3     93400.0
4     96500.0
dtype: float64

In [16]:
rmse_on_subset_tree= np.sqrt(mean_squared_error(some_labels, subset_of_predictions_tree))
rmse_on_subset_tree

0.0

Wow!!!!????, is this possible? maybe, let's take a look to all data.

In [17]:
# CODE HERE
predictions_tree= decision_tree.predict(housing_prepared)
RMSE_tree = np.sqrt(mean_squared_error(housing_labels, predictions_tree))
RMSE_tree

0.0

# Explaine this result 
write down your answar

your answer goes here

There is no error in the data? all the predictions has the same value of the actual values? there is no way, this is much more likely that the Decision Tree Regression model has badly overfit the data, unlike a Linear Regression model which is underfitting the training data, so the Linear reg model is not the best to predicte the housing price until now, so we must be sure from the two model. Now, because we don't want to use the testing data before make sure from the model that we're going to use, so we're going to use a two subsets of training data, one for training and one for model validation.

# Evaluation Using Cross-Validation

1-split the training set into 10 distinct subsets then train and evaluate the Decision Tree model

In [18]:
from sklearn.model_selection import cross_val_score

In [19]:
cross_val_score?

In [20]:
# CODE HERE
cross_val_scores = cross_val_score(decision_tree, housing_prepared, housing_labels, scoring='neg_mean_squared_error', cv=10)
cross_val_scores

array([-4.15044178e+09, -4.93471243e+09, -4.88853736e+09, -5.08419544e+09,
       -5.39475772e+09, -4.55331884e+09, -4.52965502e+09, -4.49447553e+09,
       -4.71997160e+09, -4.83182030e+09])

the scoring function is actually the opposite of the MSE (i.e., a neg‐
ative value), because of that we're going to use (- cross_val_scores) to get positive values or the mse values.

In [21]:
newTree_rmse_scores= np.sqrt(- cross_val_scores) 
newTree_rmse_scores

array([64423.92241032, 70247.50838848, 69918.07609215, 71303.54440602,
       73449.01439919, 67478.28420891, 67302.71186049, 67040.84969508,
       68702.04942851, 69511.29623638])

2- display the resultant scores and calculate its Mean and Standard deviation

In [22]:
# CODE HERE
print('Cross validation for (Decision tree)scores: ', newTree_rmse_scores)
print('Mean of scoes: ', newTree_rmse_scores.mean())
print('Standard deviation of scoes: ', newTree_rmse_scores.std())

Cross validation for (Decision tree)scores:  [64423.92241032 70247.50838848 69918.07609215 71303.54440602
 73449.01439919 67478.28420891 67302.71186049 67040.84969508
 68702.04942851 69511.29623638]
Mean of scoes:  68937.72571255386
Standard deviation of scoes:  2403.8670171623703


Now this is possible, it is a possible number 68454 with std +/- 2460, let's take a look to the linear reg model with cross val maybe we can get a best value.

3-repaet the same steps to compute the same scores for the Linear Regression  model 

*notice the difference between the results of the two models*

In [23]:
# CODE HERE
cross_val_scores_2 = cross_val_score(linear_reg, housing_prepared, housing_labels, scoring='neg_mean_squared_error', cv=10)
cross_val_scores_2

array([-4.22508760e+09, -5.03540116e+09, -4.50544871e+09, -4.36783940e+09,
       -4.67890842e+09, -4.25969610e+09, -4.25348949e+09, -4.69574001e+09,
       -5.29108950e+09, -4.75511489e+09])

In [24]:
newLinear_rmse_scores= np.sqrt(- cross_val_scores_2) 
newLinear_rmse_scores

array([65000.67382615, 70960.56056304, 67122.63935124, 66089.63153865,
       68402.54686442, 65266.34735288, 65218.78174481, 68525.46981754,
       72739.87555996, 68957.34111906])

In [25]:
print('Cross validation (for Linear reg) scores: ', newLinear_rmse_scores)
print('Mean of scoes: ', newLinear_rmse_scores.mean())
print('Standard deviation of scoes: ', newLinear_rmse_scores.std())

Cross validation (for Linear reg) scores:  [65000.67382615 70960.56056304 67122.63935124 66089.63153865
 68402.54686442 65266.34735288 65218.78174481 68525.46981754
 72739.87555996 68957.34111906]
Mean of scoes:  67828.38677377408
Standard deviation of scoes:  2468.091395065227


Ok, 67828 with +/- 2468 std is the best until now, so the linear reg perform is better than decision tree model,

## Let’s train one last model the RandomForestRegressor.

In [26]:
# CODE HERE
from sklearn.ensemble import RandomForestRegressor

RFG = RandomForestRegressor()
RFG.fit(housing_prepared, housing_labels)

RandomForestRegressor()

In [27]:
predictions_RFG= RFG.predict(housing_prepared)
predictions_RFG

array([109940.  , 383497.06, 194275.  , ..., 212016.  , 276375.  ,
       317583.01])

In [28]:
rmse_on_RFG= np.sqrt(mean_squared_error(housing_labels, predictions_RFG))
rmse_on_RFG

18609.799683094494

Wow, that is possible 18525 is the best answer, so the RFG looks so promising.  

But we must use cross validation to know if the model is overfitted in training set of data (that is so possible), so let's use cross_val on RFG to see the results.

In [29]:
RFG_2 = RandomForestRegressor(n_estimators=100, random_state=42)
RFG_2.fit(housing_prepared, housing_labels)

RandomForestRegressor(random_state=42)

In [30]:
predictions_RFG_2= RFG_2.predict(housing_prepared)
predictions_RFG_2

array([107699.  , 384707.07, 206139.03, ..., 211776.  , 278064.  ,
       316979.02])

In [31]:
rmse_on_RFG= np.sqrt(mean_squared_error(housing_labels, predictions_RFG_2))
rmse_on_RFG

18527.322990316152

# repeat the same steps to compute the same scores its Mean and Standard deviation for the Random Forest model

In [32]:
# CODE HERE
cross_val_scores_3 = cross_val_score(RFG, housing_prepared, housing_labels, scoring='neg_mean_squared_error', cv=10)
cross_val_scores_3

array([-2.24107511e+09, -2.68679743e+09, -2.46637925e+09, -2.67323858e+09,
       -2.80480508e+09, -2.23016627e+09, -2.23534784e+09, -2.58508388e+09,
       -2.42701188e+09, -2.48153083e+09])

In [33]:
newRFG_rmse_scores= np.sqrt(- cross_val_scores_3) 
newRFG_rmse_scores

array([47339.99479348, 51834.32672299, 49662.65447745, 51703.37102818,
       52960.41047389, 47224.63629308, 47279.46527992, 50843.72017669,
       49264.71227652, 49814.96596071])

In [34]:
print('Cross validation (for Linear reg) scores: ', newRFG_rmse_scores)
print('Mean of scoes: ', newRFG_rmse_scores.mean())
print('Standard deviation of scoes: ', newRFG_rmse_scores.std())

Cross validation (for Linear reg) scores:  [47339.99479348 51834.32672299 49662.65447745 51703.37102818
 52960.41047389 47224.63629308 47279.46527992 50843.72017669
 49264.71227652 49814.96596071]
Mean of scoes:  49792.825748290874
Standard deviation of scoes:  1954.000455277528


This is much better than other models, with cross_val there isn't overfitting.

# Save every model you experiment with 
*using the joblib library*

In [35]:
# CODE HERE
import joblib
joblib.dump(linear_reg, "linear regression.pkl")
joblib.dump(decision_tree, "decision tree.pkl")
joblib.dump(RFG, "random forest regressor.pkl")
joblib.dump(RFG_2, "random forest regressor_100 estimators.pkl")

['random forest regressor_100 estimators.pkl']

## now you have a shortlist of promising models. You now need to
## fine-tune them!
# Fine-Tune Your Model

## 1- Grid Search
## evaluate all the possible combinations of hyperparameter values for the RandomForestRegressor 
*It may take a long time*

In [36]:
from sklearn.model_selection import GridSearchCV

In [37]:
# CODE HERE
hyperpars = [
                    {'n_estimators': [3, 10, 30, 40], 'max_features': [2, 4, 6, 8]},
                    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},# bootstrap: False, the boot strap is used in applied machine learning to estimate 
                                                                                                                               #the skill of machine learning models when making predictions on data not included in the training data.
                    {'bootstrap': [False], 'n_estimators': [3, 10, 30], 'max_features': [2, 3, 4, 6, 8]}
]

with the evaluation scores

In [38]:
# CODE HERE
grid_search = GridSearchCV(RFG, hyperpars, cv=5,
                                                scoring='neg_mean_squared_error',
                                                return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30, 40]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4, 6, 8],
                          'n_estimators': [3, 10, 30]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [39]:
grid_search.best_params_

{'bootstrap': False, 'max_features': 6, 'n_estimators': 30}

In [40]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=False, max_features=6, n_estimators=30)

In [41]:
grid_search.cv_results_['mean_test_score']

array([-4.04456322e+09, -2.99068583e+09, -2.72519212e+09, -2.70980324e+09,
       -3.52848928e+09, -2.77006078e+09, -2.53672912e+09, -2.48329610e+09,
       -3.53753899e+09, -2.69555346e+09, -2.51119655e+09, -2.45254166e+09,
       -3.45142818e+09, -2.71474686e+09, -2.47563867e+09, -2.48197048e+09,
       -4.01432791e+09, -2.91077099e+09, -3.60539854e+09, -2.74486141e+09,
       -3.34508623e+09, -2.60913650e+09, -3.82930745e+09, -2.93336438e+09,
       -2.63868361e+09, -3.47744129e+09, -2.75648934e+09, -2.50468298e+09,
       -3.30379584e+09, -2.63858692e+09, -2.43045784e+09, -3.28315042e+09,
       -2.59926773e+09, -2.38884976e+09, -3.28340454e+09, -2.63067441e+09,
       -2.43763160e+09])

In [42]:
grid_search.cv_results_['params']

[{'max_features': 2, 'n_estimators': 3},
 {'max_features': 2, 'n_estimators': 10},
 {'max_features': 2, 'n_estimators': 30},
 {'max_features': 2, 'n_estimators': 40},
 {'max_features': 4, 'n_estimators': 3},
 {'max_features': 4, 'n_estimators': 10},
 {'max_features': 4, 'n_estimators': 30},
 {'max_features': 4, 'n_estimators': 40},
 {'max_features': 6, 'n_estimators': 3},
 {'max_features': 6, 'n_estimators': 10},
 {'max_features': 6, 'n_estimators': 30},
 {'max_features': 6, 'n_estimators': 40},
 {'max_features': 8, 'n_estimators': 3},
 {'max_features': 8, 'n_estimators': 10},
 {'max_features': 8, 'n_estimators': 30},
 {'max_features': 8, 'n_estimators': 40},
 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3},
 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10},
 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3},
 {'bootstrap': False, 'max_features': 3, 'n_estimators': 10},
 {'bootstrap': False, 'max_features': 4, 'n_estimators': 3},
 {'bootstrap': False, 'ma

In [43]:
cvres = grid_search.cv_results_

In [44]:
RMSE_list = []
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)
    RMSE_list.append(np.sqrt(-mean_score))

63596.88056153428 {'max_features': 2, 'n_estimators': 3}
54687.16330485827 {'max_features': 2, 'n_estimators': 10}
52203.37271131242 {'max_features': 2, 'n_estimators': 30}
52055.77051582462 {'max_features': 2, 'n_estimators': 40}
59401.088164507135 {'max_features': 4, 'n_estimators': 3}
52631.36684941375 {'max_features': 4, 'n_estimators': 10}
50365.95194491008 {'max_features': 4, 'n_estimators': 30}
49832.68105610091 {'max_features': 4, 'n_estimators': 40}
59477.21402158825 {'max_features': 6, 'n_estimators': 3}
51918.71978016918 {'max_features': 6, 'n_estimators': 10}
50111.84044103815 {'max_features': 6, 'n_estimators': 30}
49523.142634453856 {'max_features': 6, 'n_estimators': 40}
58748.85685895614 {'max_features': 8, 'n_estimators': 3}
52103.23271548502 {'max_features': 8, 'n_estimators': 10}
49755.790331349795 {'max_features': 8, 'n_estimators': 30}
49819.37856804238 {'max_features': 8, 'n_estimators': 40}
63358.72403596912 {'bootstrap': False, 'max_features': 2, 'n_estimators':

In [45]:
min(RMSE_list) #{'bootstrap': False, 'max_features': 6, 'n_estimators': 3}

48875.86072917433

# Analyze the Best Models and Their Errors
1-indicate the relative importance of each attribute

In [46]:
# CODE HERE
#Let's display the cols by the importance
cols_importance = grid_search.best_estimator_.feature_importances_
cols_importance

array([8.50352837e-02, 7.44107728e-02, 3.98391928e-02, 1.61368225e-02,
       1.55956669e-02, 1.61933959e-02, 1.50336439e-02, 3.33646019e-01,
       5.82734358e-02, 1.03443021e-01, 8.26589363e-02, 1.12803953e-02,
       1.37985334e-01, 2.30783671e-04, 4.19953967e-03, 6.03775797e-03])

2-display these importance scores next to their corresponding attribute names:

In [50]:
# CODE HERE
#let's use feature engineering to add 3 more cols
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]

In [53]:
full_pipeline.named_transformers_

{'num': Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                 ('attribs_adder', CombinedAttributesAdder()),
                 ('std_scaler', StandardScaler())]), 'cat': OneHotEncoder()}

In [54]:
cat_ohe = full_pipeline.named_transformers_['cat']

In [55]:
list(cat_ohe.categories_[0])

['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN']

In [60]:
cat_cols = list(cat_ohe.categories_[0])
cols = num_attribs + extra_attribs + cat_cols
cols

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'rooms_per_hhold',
 'pop_per_hhold',
 'bedrooms_per_room',
 '<1H OCEAN',
 'INLAND',
 'ISLAND',
 'NEAR BAY',
 'NEAR OCEAN']

In [61]:
sorted(zip(cols_importance, cols), reverse=True)

[(0.3336460185891032, 'median_income'),
 (0.13798533356056822, 'INLAND'),
 (0.10344302061657772, 'pop_per_hhold'),
 (0.08503528371161687, 'longitude'),
 (0.08265893627515516, 'bedrooms_per_room'),
 (0.07441077278660943, 'latitude'),
 (0.058273435756137554, 'rooms_per_hhold'),
 (0.03983919281989622, 'housing_median_age'),
 (0.01619339593868997, 'population'),
 (0.016136822518739366, 'total_rooms'),
 (0.015595666912119915, 'total_bedrooms'),
 (0.015033643929227652, 'households'),
 (0.011280395266076085, '<1H OCEAN'),
 (0.006037757973569321, 'NEAR OCEAN'),
 (0.004199539674874237, 'NEAR BAY'),
 (0.00023078367103913341, 'ISLAND')]

median income the most impotant column, it is expected.
we notice here the new column population per house hold is the third column by importance and maybe this will improve the model performance, and the only useful category from the ocean prox column is the INLAND others are not useful so we can drop it .

## Now is the time to evaluate the final model on the test set.
# Evaluate Your System on the Test Set

1-get the predictors and the labels from your test set

In [70]:
# CODE HERE
model = grid_search.best_estimator_

In [69]:
predictors_test = test_set.drop("median_house_value", axis=1)
label_test = test_set["median_house_value"].copy()

2-run your full_pipeline to transform the data

In [78]:
# CODE HERE
test_prepared = full_pipeline.transform(predictors_test)
real_predictions = model.predict(test_prepared)
real_predictions

array([ 48080.        , 103613.33333333, 423963.8       , ...,
       496597.6       ,  74026.66666667, 171713.33333333])

3-evaluate the final model on the test set

In [77]:
# CODE HERE
final_mse = mean_squared_error(label_test, real_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

48372.78019350712

# compute a 95% confidence interval for the generalization error 
*using scipy.stats.t.interval():*

here we must have an idea of how precise this estimate is.

In [79]:
from scipy import stats

In [80]:
# CODE HERE
from scipy import stats

confidence = 0.95
squared_errors = (real_predictions - label_test) ** 2

np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                                   loc=squared_errors.mean(),
                                   scale=stats.sem(squared_errors)))

array([46181.18040205, 50469.30060911])

# Great Job!
# #shAI_Club